Verify Python environment

In [1]:
import sys
print(sys.executable)

c:\Users\alals\repos\perf-pilot\venv\Scripts\python.exe


List installed packages

In [ ]:
%pip list

Import dependencies

In [24]:
import requests
from pprint import pprint
import csv

API preparations

In [25]:
# Bugzilla API key
API_KEY = "sicSwVcOxOwvLCn12804YSB0nhlCuSBw4FkhKoKK"
BUGZILLA_API_URL = "https://bugzilla.mozilla.org/rest"


Get performance bugs that are also regressions

In [23]:
ENDPOINT = f"{BUGZILLA_API_URL}/bug"

FIELDS = ["id", "regressed_by"]

params = {
        "api_key": API_KEY,
        "keywords": ["perf"],
        "limit": 5,
        "f1": "regressed_by",
        "o1": "isnotempty",
        "include_fields": ",".join(FIELDS),
        "offset": 1
    }
    
response = requests.get(ENDPOINT, params=params)

# response.json()

bugs = response.json().get("bugs", [])

for bug in bugs:
            print(f"#{bug['id']}:")
            print(f"  Regressed by: {bug['regressed_by']}")
            print()
    

#1533662:
  Regressed by: [1529340]

#1535257:
  Regressed by: [1534444]

#1541314:
  Regressed by: [1524665]

#1543403:
  Regressed by: [1538248]

#1544369:
  Regressed by: [1496075]



Get bugs and save them to a csv file

In [27]:
BUGZILLA_API = f"{BUGZILLA_API_URL}/bug"
FIELDS = ["id", "summary", "regressed_by"]
LIMIT = 100  # Page size
MAX_TOTAL = 1000  # Cap on total bugs to fetch

def fetch_all_regressed_bugs():
    offset = 0
    all_bugs = []

    while len(all_bugs) < MAX_TOTAL:
        remaining = MAX_TOTAL - len(all_bugs)
        page_limit = min(LIMIT, remaining)

        params = {
            "api_key": API_KEY,
            "include_fields": ",".join(FIELDS),
            "f1": "regressed_by",
            "o1": "isnotempty",
            "limit": page_limit,
            "offset": offset
        }

        response = requests.get(BUGZILLA_API, params=params)
        if response.status_code != 200:
            print(f"Failed to fetch bugs: {response.status_code}")
            print(response.text)
            break

        bugs = response.json().get("bugs", [])
        if not bugs:
            break  # No more bugs

        all_bugs.extend(bugs)
        offset += page_limit

    print(f"✅ Total bugs fetched: {len(all_bugs)}")
    return all_bugs

def save_bugs_to_csv(bugs, filename="regressed_bugs.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(FIELDS)  # Header

        for bug in bugs:
            writer.writerow([
                bug.get("id"),
                bug.get("summary"),
                ",".join(str(x) for x in bug.get("regressed_by", []))
            ])

    print(f"📁 Bugs saved to {filename}")



bugs = fetch_all_regressed_bugs()
save_bugs_to_csv(bugs)


✅ Total bugs fetched: 1000
📁 Bugs saved to regressed_bugs.csv


Get bug ticket attachments for a single bug

In [30]:
ENDPOINT = "https://bugzilla.mozilla.org/rest/bug/1947908/attachment"

FIELDS = ["id", "regressed_by"]

params = {
        "api_key": API_KEY,
        "keywords": ["perf"],
        "limit": 5,
        "f1": "regressed_by",
        "o1": "isnotempty",
        "include_fields": ",".join(FIELDS),
        "offset": 1
    }
    
response = requests.get(ENDPOINT)

pprint(response.json())

{'attachments': {},
 'bugs': {'1947908': [{'attacher': 'brennie@mozilla.com',
                       'bug_id': 1947908,
                       'content_type': 'text/x-phabricator-request',
                       'creation_time': '2025-03-05T05:50:32Z',
                       'creator': 'brennie@mozilla.com',
                       'creator_detail': {'email': 'brennie@mozilla.com',
                                          'id': 529428,
                                          'name': 'brennie@mozilla.com',
                                          'nick': 'beth',
                                          'real_name': 'Beth Rennie [:beth] '
                                                       '(she/her)'},
                       'data': 'aHR0cHM6Ly9waGFicmljYXRvci5zZXJ2aWNlcy5tb3ppbGxhLmNvbS9EMjQwNDE4',
                       'description': 'Bug 1947908 - Remove update:slug event '
                                      'from ExperimentStore '
                                      'r?

- Get all the bugs in the last year
- get all the bugs from regressions.csv from treeherder module
- if a bug is a regression bug its "regressed by" field is empty, remove that bug from both regressions.csv and all the bugs extracted in the first step. TODO: we could employ SZZ here instead of removing the bug but that's for the future.
- For each regression bug, inspect its "regressed by" field and create a new list of bugs

- For each bug/ticket, have these columns bug id, summary, caused perf regression, product, component
- Take regressed_by.csv from the treeherder module that has a list of bug/ticket ids
- For each bug, if the bug also exists in regressed_by.csv, mark the "caused perf regression" as true
- create a list of perf-related products and components from the regressions.csv file's bugs
- from regressed_by.csv file, remove bugs that are not in products and components gathered above

